# NIRISS AMI: MIRAGE Simulations

**Use case:** Simulation of NIRISS AMI data.<br>
**Data:** JWST simulated NIRISS data from MIRAGE.<br>
**Tools:**  mirage, jwst, astropy.<br>
**Cross-intrument:** NIRCam. <br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

## Introduction

This notebook creates MIRAGE simulations for binary point source AB Dor and calibrator HD37093 using the Aperture Masking Interferometry (AMI) mode on JWST NIRISS. The sources are observed with the Non-redundant mask (NRM) in the pupil wheel and filter F480M in the filter wheel at POS1 dither position near the center of SUB80 array. The goal of the program is to find the binary parameters of AB Dor.

The current analysis tools are work in progress and do not give correct results if the data has bad pixels. The simulations are therefore created using dark ramps without bad pixels.

### Description of reference files used for simulations

We are using the following non-default reference files to generate the simulation. 
- dark000001_uncal.fits has ngroups = 5 and is used for AB Dor observation that also has ngroups = 5. 
- dark000005_uncal.fits has ngroups = 12 and is used for HD37093 observation that also has ngroups = 12. 
- jwst_niriss_gain_general.fits is the gain file with one value for all pixels 
- jwst_niriss_flat_general.fits is the flat field reference file with no structure 
- jwst_niriss_superbias_sim.fits bias reference file to match the simulation

We are temporarily using a local niriss_gridded_psf_library that was created using a new version of non redundant mask. When Mirage gridded psf library gets updated with WebbPSF PSFs that use new version of NRM we can switch back to the default library.

*Developer Note:*
Create the conda environment needed to run this notebook by issuing the following command:
```
conda create -n <myenv> python=3.6
```
Activate the environment and then open the notebook.

```
conda activate <myenv>
jupyter notebook
```

***

## Imports
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *matplotlib.pyplot* for plotting data
- *zipfile* for accessing zip files
- *urllib.request* to access URL
- *yaml* to create yaml files
- *mirage* to simulate JWST data
- *IPython.display Image* to display png files

In [ ]:
import glob
import io
import os
import sys

import numpy as np
from astropy.io import fits
import yaml
import zipfile
import urllib.request
from IPython.display import Image

from mirage import imaging_simulator
from mirage.yaml import yaml_generator

%matplotlib inline
import matplotlib.pyplot as plt

<div class="alert alert-info">

**Note:** DO NOT UPGRADE PYSIAF AS INSTRUCTED BY THE ABOVE WARNING. 
Pysiaf 0.10.0 uses PRD release PRDOPSSOC-031. APT 2020.4.1 that was used to create the xml and pointing files used in this notebook uses PRDOPSSOC-030. The mismatch bewtween these two PRD versions causes incorrect placement of PSF. The version of Mirage used in this notebook comes with pysiaf 0.10.0. It was downgraded to 0.9.0 using the file requirements.txt to resolve the placement issue. Using Pysiaf 0.10.0 will cause incorrect placement of the PSF.

</div>

Mirage is accompanied by a set of reference files that are used to construct simulated data. They are specified by MIRAGE_DATA environment variable. These files include dark current ramps, cosmic ray and PSF libraries.

*Developer Note:*
If you are outside STScI install the mirage data by following instructions on https://mirage-data-simulator.readthedocs.io/en/latest/reference_files.html and create MIRAGE_DATA location.

In [ ]:
if os.environ.get('MIRAGE_DATA', None) is None:
    os.environ['MIRAGE_DATA'] = '/path/to/mirage_data/'
print(os.environ.get('MIRAGE_DATA'))

### Loading input files and reference files

In [ ]:
boxlink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/niriss_ami_binary/niriss_ami_binary1.zip'
boxfile = './niriss_ami_binary1.zip'

# Download zip file
if not os.path.exists(boxfile):
    print("Downloading input files needed to run the notebook. This may take some time")
    urllib.request.urlretrieve(boxlink, boxfile)

    zf = zipfile.ZipFile(boxfile, 'r')
    zf.extractall()
else:
    print("Input files exist. You may want to check if there is an updated version.")

### Create output directory to store simulated data

In [ ]:
odir = './mirage_sim_data'
if not os.path.exists(odir):
    os.mkdir(odir)
simdata_output_directory = odir

### Generating input yaml files

Begin working on the simulation starting with the APT file. The xml and pointings files must be exported from APT by using 'File ---> Export' option in APT. These files are then used as input to the yaml_generator, that generates yaml input files for each exposure.

We are including the xml and pointing files along with other file included in the zip file.



In [ ]:
# Write down the name of the APT file in comments for quick reference and also store it in the same folder as xml and 
# pointing files for easy access and to remember which APT file was used for simulations.
# APT file used niriss_ami_binary_2022.25coords.aptx 
xml_name = './mirage_input_files/niriss_ami_binary_2022.25coords.xml'
pointing_name = './mirage_input_files/niriss_ami_binary_2022.25coords.pointing'

We will generate NIRISS AMI simulations of a binary point source using the catalogue "stars_field19_20_combined_allfilters_new.list". This catalogue contains the coordinates and magnitudes of AB Dor and HD37093 along with other fainter sources in the field.

***
<span style="color:darkblue">NOTE: Mirage currently does not apply the proper motion that is entered in the APT fie. It is therefore important to enter coordinates at the epoch of observation in the APT file. AB Dor is a high proper motion star so we are using 2022.25 coordinates in the APT file and the input source list file.</span>.
***

The first line in this file is for AB-Dor primary star and the second line is the faint companion that we are trying to detect. Several lines below is the reference star or calibrator star HD37093.
```
# 
# vegamag
# 
# 
x_or_RA y_or_Dec niriss_f090w_magnitude niriss_f115w_magnitude niriss_f140m_magnitude niriss_f150w_magnitude niriss_f158m_magnitude niriss_f200w_magnitude niriss_f277w_magnitude niriss_f356w_magnitude niriss_f380m_magnitude niriss_f430m_magnitude niriss_f444w_magnitude niriss_f480m_magnitude
  82.18740518  -65.44767541   5.88850   5.49770   5.07560   4.95800   4.83650   4.72940   4.72220   4.61000   4.61000   4.61000   4.61000   4.61000
  82.18717120  -65.44764863  12.06621  10.73581  10.47740  10.15193   9.84442   9.76398   9.75940   8.99121   8.69367   8.76689   8.81310   8.81310
  82.18534722  -65.44612065   8.88930   8.38010   7.86780   7.70440   7.54010   7.38740   7.38650   7.29090   7.28470   7.33890   7.38820   7.49960
  82.19501500  -65.44532800  12.13360  12.89690  12.47030  12.40400  12.28660  13.19490  14.01950  14.60890  14.67080  15.09080  15.20730  15.56880
  82.19343600  -65.45299500  13.54130  13.63920  13.68400  13.69520  13.70530  13.74310  13.76640  13.78850  13.78900  13.79520  13.79330  13.79180
  
...more sources...

# 
# vegamag
# 
# 
#x_or_RA y_or_Dec niriss_f090w_magnitude niriss_f115w_magnitude niriss_f140m_magnitude niriss_f150w_magnitude niriss_f158m_magnitude niriss_f200w_magnitude niriss_f277w_magnitude niriss_f356w_magnitude niriss_f380m_magnitude niriss_f430m_magnitude niriss_f444w_magnitude niriss_f480m_magnitude
  	
  82.78450088  -65.12833088   7.33570   6.74320   6.18410   5.99130   5.80110   5.61850   5.62170   5.49700   5.49700   5.53100   5.53100   5.53100

```

In [ ]:
catalogues = {'AB-DOR': {'point_source':  './mirage_input_files/stars_field19_20_combined_allfilters_new.list'
                        },
              'HD-37093': {'point_source': './mirage_input_files/stars_field19_20_combined_allfilters_new.list'
                          }
             }

### Set the telescope roll angle PAV3 for each observation. Another way to get PAV3 is from the APT file Reports.

In [ ]:
obs1 = 1
pav3_obs1 = yaml_generator.default_obs_v3pa_on_date(pointing_name, obs1, date='2022-04-01')
obs2 = 2
pav3_obs2 = yaml_generator.default_obs_v3pa_on_date(pointing_name, obs2, date='2022-04-01')

roll_angle = {'001': pav3_obs1, '002': pav3_obs2}

dates = '2022-04-01'
reffile_defaults = 'crds'
datatype = 'raw'
print("PAV3 for observation 1", pav3_obs1, "PAV3 for observation 2", pav3_obs2)

### Run the yaml generator 

This will create two yaml files that will be used as inputs when creating the simulated data. There will be one yaml file for each exposure in an observation. In our case we have one F480M exposure per observation.


In [ ]:
# Delete yaml files created in an earlier run
old_yaml_files = glob.glob(os.path.join(odir, 'jw*.yaml'))
for oldf in old_yaml_files:
    os.remove(oldf)

In [ ]:
yam = yaml_generator.SimInput(input_xml=xml_name, pointing_file=pointing_name,
                              catalogs=catalogues, roll_angle=roll_angle,
                              dates=dates, reffile_defaults=reffile_defaults,
                              verbose=True, output_dir=odir,
                              simdata_output_dir=simdata_output_directory,
                              datatype=datatype)

yam.create_inputs()
print("Created yaml files")

# Create yaml files for all observations.
yaml_files = sorted(glob.glob(os.path.join(odir, 'jw*.yaml')))

print(yaml_files)

### Update the contents of yaml files and generate raw data

Updating the yaml files is not always required. We are doing it here to generate data without bad pixels and to make a few other modifications.

In [ ]:
for file in yaml_files:

    # set astrometric reference file to None to use pysiaf
    # To create data without bad pixels use the following reference files.
    with open(file, 'r') as infile:
        yaml_content = yaml.safe_load(infile)
    yaml_content['Reffiles']['astrometric'] = 'None'
    yaml_content['simSignals']['psf_wing_threshold_file'] = 'config'
    yaml_content['Reffiles']['linearized_darkfile'] = 'None'
    yaml_content['simSignals']['psfpath'] = './ref_files_non_default/niriss_gridded_psf_library_newmask'
    yaml_content['Reffiles']['gain'] = './ref_files_non_default/jwst_niriss_gain_general.fits'
    yaml_content['Reffiles']['pixelflat'] = './ref_files_non_default/jwst_niriss_flat_general.fits'
    yaml_content['Reffiles']['superbias'] = './ref_files_non_default/jwst_niriss_superbias_sim.fits'

    if "jw01093001" in file:
        yaml_content['Reffiles']['dark'] = './ref_files_non_default/simdarks/dark000001/dark000001_uncal.fits'
    elif "jw01093002" in file:
        yaml_content['Reffiles']['dark'] = './ref_files_non_default/simdarks/dark000005/dark000005_uncal.fits'

    modified_file = file.replace('.yaml', '_mod.yaml')
    with io.open(modified_file, 'w') as outfile:
        yaml.dump(yaml_content, outfile, default_flow_style=False)

    print("Updated yaml files. The jw*_mod.yaml files will be used to create data")

    # create data
    t1 = imaging_simulator.ImgSim()
    t1.paramfile = str(modified_file)
    t1.create()

## Useful output products generated by Mirage

```
- jw01093001001_01101_00001_nis_uncal_pointsources.list and 
  jw01093002001_01101_00001_nis_uncal_pointsources.list. The first line of 
  jw01093001001_01101_00001_nis_uncal_pointsources.list shows coordinates of AB Dor, the
  pixel coordinates at which the data is simulated, magnitude and total count rate.
   
- jw01093001001_01101_00001_nis_uncal_F480M_NRM_final_seed_image.fits and  
  jw01093002001_01101_00001_nis_uncal_F480M_NRM_final_seed_image.fits
  A seed image is a noiseless image that contains signal only from simulated 
  astronomical sources. It can be used for quality checks on the final output data.

- jw01093001001_01101_00001_nis_uncal.fits and jw01093002001_01101_00001_nis_uncal.fits
  AB Dor and HD37093 raw data
```

### Examine the seed images

In [ ]:
data_seed = []
data_seed = []
seed_images = sorted(glob.glob('mirage_sim_data/jw*final_seed_image.fits'))
for i, df in enumerate(seed_images):
    seed_im = fits.open(df)
    seed_im.info()
    im = seed_im[1].data
    print(im.shape)
    data_seed.append(im)
print(data_seed[0].shape, data_seed[1].shape)

#Temporarily adding the two lines below. 
#Plots are rendered only when cell 1 is run twice.
import matplotlib
%matplotlib inline

f = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("AB-Dor seed image")
plt.imshow(data_seed[0], origin='lower')
plt.draw()
plt.subplot(1, 2, 2)
plt.title("HD37093 seed image")
plt.imshow(data_seed[1], origin='lower')

In [ ]:
def find_location_of_peak(image):
    """ Find the location of PSF peak when bad pixels are not present"""
    peak_location = np.where(image == image.max())
    y_peak = peak_location[0][0]
    x_peak = peak_location[1][0]
    print("x_peak_python,y_peak_python", x_peak, y_peak)
    y_peak_ds9 = y_peak + 1
    x_peak_ds9 = x_peak + 1
    print("x_peak_ds9,y_peak_ds9", x_peak_ds9, y_peak_ds9)
    return x_peak, y_peak

POS1 = find_location_of_peak(data_seed[0])
print(POS1)

if POS1 != (45, 40):
    print('****************WARNING: PSF placement is not correct****************')

#### Compare peak pixel count rate for AB-Dor seed image with peak count rate in equivalent JWST ETC calculation


In [ ]:
print("Pixel count rate for AB-Dor simulated image", data_seed[0].max(), "ADU/s")
print("Pixel count rate for AB-Dor simulated image", data_seed[0].max() * 1.61, "electrons/s")
# Upload screenshot from the ETC workbook.
from IPython.display import Image
Image("AB_Dor_ngroup5_nint65_F480M_jwetc_calc.png", width=500, height=500)

Comparison of the peak pixel count rate for AB-Dor simulated data with an equivalent JWST ETC calculation shows that the ETC peak pixel count rate of 71388.43 electrons/sec closely matches Mirage simulation. 

### Examine output point source list file 'jw01093001001_01101_00001_nis_uncal_pointsources.list'


```
# Field center (degrees):   82.18741002   -65.44767991 y axis rotation angle (degrees): 108.118419  image size: 0080 0080
#
#    Index   RA_(hh:mm:ss)   DEC_(dd:mm:ss)   RA_degrees      DEC_degrees     pixel_x   pixel_y    magnitude   counts/sec    counts/frame    TSO_lightcurve_catalog
1 05:28:44.9772 -65:26:51.6315    82.18740518   -65.44767541    45.198    39.816      4.610   1.797615e+07    1.356121e+06  None
2 05:28:44.9211 -65:26:51.5351    82.18717120   -65.44764863    44.842    34.305      8.813   3.745042e+05    2.825260e+04  None
3 05:28:44.4833 -65:26:46.0343    82.18534722   -65.44612065   110.711   -31.926      7.500   1.255616e+06    9.472364e+04  None
5 05:28:46.4246 -65:27:10.7820    82.19343600   -65.45299500  -186.533   263.381     13.792   3.819238e+03    2.881233e+02  None
```

It would be worth noting the expected magnitude contrast in F480M is 4.2 magnitudes, and the pixel offset of the faint companion is -0.356, -5.511 pixels in x and y based on the output point source list.

### Compare total count rate in the seed image with total count rate in the output pointsource list file.
```
counts/sec in jw01093001001_01101_00001_nis_uncal_pointsources.list 
is 1.797615e+07 ADU/sec

Throughput of NRM is ~0.155
counts/sec with non redundant mask (NRM) = 1.797615e+07 ADU/sec * 0.155 
                                         = 2786303.25 ADU/sec
                                         
The total count rate in the seed image is 2662913.1485110847 ADU/sec. The seed image is 80x80 pixels and 
therefore SUB80 observations are subject to aperture losses that cause this discrepancy.
```


### Examine the raw data

In [ ]:
datafiles = sorted(glob.glob('mirage_sim_data/jw*uncal.fits'))
print(datafiles)

In [ ]:
data = []
for i, df in enumerate(datafiles):
    file = fits.open(df)
    file.info()
    im = file[1].data
    print(im[0].shape)
    data.append(im[0])
print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("AB-Dor")
plt.imshow(data[0][4], origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD37093")
plt.imshow(data[1][11], origin='lower')

## Calibrate raw data (_uncal.fits files) with the JWST pipiline

Use 2_niriss_ami_binary.ipynb to calibrate the data with JWST pipeline. 

## Aditional Resources

- [NIRISS AMI JDox](https://jwst-docs.stsci.edu/near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-aperture-masking-interferometry)


## About this notebook

**Author:** Deepashri Thatte, Kevin Volk
**Updated On:** 2020-12-18

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 